# QWEN-2.5-instruct Neuron 모델을 SageMaker Endpoint INF2 에 배포하기

이 노트북은 [Qwen/Qwen2.5-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-7B-Instruct) 의 모델을 AWS Neuron Model 로 바꾼 [Gonsoo/AWS-HF-optimum-neuron-0-0-28-Qwen2.5-7B-Instruct](https://huggingface.co/Gonsoo/AWS-HF-optimum-neuron-0-0-28-Qwen2.5-7B-Instruct/tree/main/compiled) 모델을 사용 합니다.<br>
AWS Hugging Face Text Generation Inference (HF TGI) 도커 컨테이너를 이용하여, SageMaker Endpoint 에 배포 합니다.

### 커널 설정
- 쥬피터 노트북의 오른쪽 상단의 conda_neuron_pytorch_p38 를 사용합니다.


---

# 0. 모델 컴피알 및 HF 올리기 ( 옵션 )

아래는 [Amazon SageMaker 를 이용하여 Amazon Inferentia2 기반 위에 한국어 파인 튜닝 모델을 서빙하기](../README.md) 의 가이드에 따른 AWS 도커 컨테이너로 아래와 같은 작업을 수행을 사전에 했습니다. 
참고 하시면 됩니다.

### 모델 컴파일

```
time docker run --entrypoint optimum-cli \
  -v $(pwd)/data:/data \
  --privileged \
  763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2 \
  export neuron \
  --model Qwen/Qwen2.5-7B-Instruct \
  --batch_size 4 \
  --sequence_length 4096 \
  --auto_cast_type bf16 \
  --num_cores 2 \
  /data/Qwen2.5-7B-Instruct-recompiled
```
### 모델 서빙
```
docker run -p 8080:8080 \
  -v $(pwd)/data:/data \
  --privileged \
  -e HF_MODEL_ID=/data/Qwen2.5-7B-Instruct-recompiled \
  -e HF_NUM_CORES=2 \
  -e HF_BATCH_SIZE=4 \
  -e HF_SEQUENCE_LENGTH=4096 \
  -e HF_AUTO_CAST_TYPE=bf16 \
  -e MAX_BATCH_SIZE=4 \
  -e MAX_INPUT_LENGTH=2048 \
  -e MAX_TOTAL_TOKENS=4096 \
  -e MESSAGES_API_ENABLED=true \
  763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2
```
### 추론 테스트
```
curl localhost:8080/v1/chat/completions \
    -X POST \
    -d '{
"model": "tgi",
"messages": [
    {
    "role": "system",
    "content": "당신은 역사 전문가 입니다."
    },
    {
    "role": "user",
    "content": "세종대왕 맥북 사건에 대해서 알려 주세요.?"
    }
],
"stream": false,
"max_tokens": 512
}' \
    -H 'Content-Type: application/json'  

```
### 모델 HF 에 올리기
```
huggingface-cli login --token $API_TOKEN

huggingface-cli upload  Gonsoo/AWS-HF-optimum-neuron-0-0-28-Qwen2.5-7B-Instruct \
./data/Qwen2.5-7B-Instruct-recompiled
```
---

## 1. 다음과 같은 패키지를 설치 합니다.



In [1]:
! pip install "sagemaker>=2.199.0" "gradio<4" transformers --upgrade --quiet
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
! pip list | grep -E "sagemaker|gradio|transformers|huggingface"

gradio                        3.50.2
gradio_client                 0.6.1
huggingface-hub               0.31.4
sagemaker                     2.243.2
sagemaker-core                1.0.29
sagemaker_pyspark             1.4.5
transformers                  4.46.3


## 2. SageMaker 세션 등 가져오기

In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = sess.boto_region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region}")


/home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/25/25 08:32:49] INFO     Found credentials from IAM Role:                                   ]8;id=119807;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=420415;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[05/25/25 08:32:50] INFO     Found credentials from IAM Role:                                   ]8;id=179288;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=672667;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=419402;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=312351;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=159044;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=466071;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker role arn: arn:aws:iam::057716757052:role/workshop-sagemaker-kfp-role2
sagemaker session region: us-west-2


## 3. Hugging Face LLM Inf2 DLC (Deep Learning Container ) 가져오기

새로운 Hugging Face TGI Neuronx DLC를 사용하여 AWS Inferentia2에서 추론을 실행할 수 있습니다. sagemaker SDK의 get_huggingface_llm_image_uri 메서드를 사용하여 원하는 backend, session, region, 그리고 version에 따라 적절한 Hugging Face TGI Neuronx DLC URI를 검색할 수 있습니다. 사용 가능한 모든 버전은 여기에서 확인할 수 있습니다.

**<u>"여기서는 2025.05.25" 현재 가장 최신인 SageMaker TGI 이미지를 사용합니다."</u>**

- 참조: [Hugging Face TGI Neuronx DLC URI](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true)



In [4]:
# TODO: Comment in when released
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
# llm_image = get_huggingface_llm_image_uri(
#   "huggingface-neuronx",
#   ## version="0.0.22"
# )

llm_image = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2"
print(f"llm image uri: \n {llm_image}")

llm image uri: 
 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2


## 4. QWEN-2.5-Instruct 모델을 inferentia2 에 배포하기


모델을 Amazon SageMaker에 배포하기 전에 TGI Neuronx 엔드포인트 구성을 정의해야 합니다. 다음과 같은 추가 매개변수를 정의해야 합니다:

- HF_NUM_CORES: 컴파일에 사용된 Neuron 코어의 수.
- HF_BATCH_SIZE: 모델 컴파일에 사용된 배치 크기.
- HF_SEQUENCE_LENGTH: 모델 컴파일에 사용된 시퀀스 길이.
- HF_AUTO_CAST_TYPE: 모델 컴파일에 사용된 자동 캐스트 유형.

또한 기존의 TGI 매개변수도 정의해야 합니다:

- HF_MODEL_ID: Hugging Face 모델 ID.
- MAX_BATCH_SIZE: 모델이 처리할 수 있는 최대 배치 크기로, 컴파일에 사용된 배치 크기와 동일합니다.
- MAX_INPUT_LENGTH: 모델이 처리할 수 있는 최대 입력 길이.
- MAX_TOTAL_TOKENS: 모델이 생성할 수 있는 최대 총 토큰 수로, 컴파일에 사용된 시퀀스 길이와 동일합니다.

### 모델 배포 파라미터 설정 및 SageMaker HuggingFaceModel 생성

In [5]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.xlarge"
health_check_timeout=900 # additional time to load the model
volume_size=64 # size in GB of the EBS volume

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "Gonsoo/AWS-HF-optimum-neuron-0-0-28-Qwen2.5-7B-Instruct",  
    "HF_NUM_CORES": "2", # number of neuron cores
    "HF_BATCH_SIZE": "4", # batch size used to compile the model
    "HF_SEQUENCE_LENGTH": "4096", # length used to compile the model
    "HF_AUTO_CAST_TYPE": "bf16",  # dtype of the model
    "MAX_BATCH_SIZE": "4", # max batch size for the model
    "MAX_INPUT_LENGTH": "2048", # max length of input text
    "MAX_TOTAL_TOKENS": "4096", # max length of generated text
    "MESSAGES_API_ENABLED": "true", # Enable the messages API
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

[05/25/25 08:32:51] INFO     Found credentials from IAM Role:                                   ]8;id=345204;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=87923;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### 모델을 SageMaker Endpoint 에 배포

HuggingFaceModel을 생성한 후에는 deploy 메서드를 사용하여 Amazon SageMaker에 배포할 수 있습니다. 우리는 ml.inf2.xlarge 인스턴스 유형을 사용하여 모델을 배포할 것입니다. TGI는 자동으로 모든 Inferentia 디바이스에 걸쳐 모델을 분산하고 샤딩할 것입니다.
- 다음은 약 17분이 소요 됩니다.

In [ ]:
%%time

# deactivate warning since model is compiled
llm_model._is_compiled_model = True

from datetime import datetime

# endpoint_name 생성 (sm-llama3-kr-inf2-yyyy-mm-dd-hh-mm-ss 형식)
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
endpoint_name = f"sm-qwen-2-5-instruct-inf2-{timestamp}"


llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  volume_size=volume_size,
  endpoint_name=endpoint_name
)

                    INFO     Creating model with name:                                              ]8;id=988030;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=966020;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-05-25-08-32-51-779                       

[05/25/25 08:32:52] INFO     Creating endpoint-config with name                                     ]8;id=312452;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=416767;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#6019\6019]8;;\
                             sm-qwen-2-5-instruct-inf2-2025-05-25-08-32-51                                         

                    INFO     Creating endpoint with name                                            ]8;id=863407;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=497687;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4841\4841]8;;\
                             sm-qwen-2-5-instruct-inf2-2025-05-25-08-32-51                                         

--------------------------!CPU times: user 435 ms, sys: 48.4 ms, total: 483 ms
Wall time: 14min 34s


## 5. 추론

엔드포인트가 배포된 후에는 이를 통해 추론을 실행할 수 있습니다. 우리는 predictor의 predict 메서드를 사용하여 엔드포인트에서 추론을 실행할 것입니다. 생성에 영향을 미치는 다양한 매개변수로 추론을 수행할 수 있습니다. 매개변수는 페이로드의 parameters 속성에서 정의할 수 있습니다. 지원되는 매개변수는 여기에서 확인할 수 있습니다.
메시지 API를 사용하면 대화 방식으로 모델과 상호작용할 수 있습니다. 메시지의 역할과 내용을 정의할 수 있습니다. 역할은 system, assistant 또는 user일 수 있습니다. system 역할은 모델에 컨텍스트를 제공하는 데 사용되고, user 역할은 질문을 하거나 모델에 입력을 제공하는 데 사용됩니다.

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What is deep learning?" }
  ]
}
```

## 5.1. Message API 로 추론

In [ ]:
# Prompt to generate
messages=[
    { "role": "system", "content": "당신은 인공지능 전문가 입니다." },
    { "role": "user", "content": "딥러닝이 무엇인지 말해 주세요?" }
  ]

# Generation arguments
parameters = {
    "model": "meta-llama/Meta-Llama-3-70B-Instruct", # placholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 2048,
    "stop": ["<|eot_id|>"],
}

이제 테스트 해보시죠

In [ ]:
chat = llm.predict({"messages" :messages, **parameters})
# chat = llm.predict({"messages" :messages, **parameters,"stream":True})

print(chat["choices"][0]["message"]["content"].strip())

딥러닝은 인공지능의 한 분야로, 이는 복잡한 데이터를 처리하고 분석하는 데 사용되는 인공신경망의 구조와 알고리즘을 활용합니다. "딥"이라는 용어는 이러한 신경망이 깊은 계층을 가질 수 있음을 의미하며, 이는 입력 데이터를 처리하고 추출하는 데 사용되는 복잡한 기능을 표현하는 데 도움이 됩니다.

딥러닝은 주로 머신 러닝에서 사용되며, 이는 컴퓨터가 데이터에서 패턴을 학습하고 이를 바탕으로 예측이나 결정을 내리는 기술입니다. 딥러닝은 이미지 및 음성 인식, 자연어 처리, 추천 시스템 등 다양한 분야에서 활용되고 있습니다.

딥러닝의 핵심은 다음과 같습니다:

1. **다중 계층**: 딥러닝 모델은 수많은 계층을 가질 수 있으며, 이는 입력 데이터를 점진적으로 더 복잡한 표현으로 변환하는 데 도움이 됩니다.
2. **학습**: 딥러닝 모델은 주어진 데이터 세트를 통해 학습됩니다. 이 과정에서 모델은 입력과 출력 간의 관계를 학습하고, 이를 통해 새로운 데이터에 대한 예측을 수행할 수 있습니다.
3. **강화 학습**: 일부 딥러닝 모델은 강화 학습이라는 개념을 사용하여, 환경과 상호작용하며 최적의 행동을 찾아냅니다.

딥러닝은 빅 데이터와 함께 발전하며, 복잡한 패턴을 인식하고 분석하는 능력으로 인해 많은 분야에서 중요한 역할을 수행하고 있습니다.<|im_end|>


## 5.2. Streaming 추론

In [ ]:
import boto3
import json

runtime = boto3.client('sagemaker-runtime')

payload = {
    "messages": messages,
    **parameters,
    "stream": True
}

response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

# 수정된 스트리밍 응답 처리
for event in response['Body']:
    try:
        # Bytes를 string으로 decode
        chunk_bytes = event['PayloadPart']['Bytes']
        chunk_str = chunk_bytes.decode('utf-8')
        
        # 빈 문자열이나 공백만 있는 경우 건너뛰기
        if not chunk_str.strip():
            continue
            
        # SSE 형태 데이터 처리 (data: 접두사 제거)
        if chunk_str.startswith('data: '):
            chunk_str = chunk_str[6:]
            
        # [DONE] 메시지는 스트림 종료 신호
        if chunk_str.strip() == '[DONE]':
            break
            
        # JSON 파싱
        chunk = json.loads(chunk_str)
        
        # OpenAI 호환 형태 처리
        if 'choices' in chunk:
            delta = chunk['choices'][0].get('delta', {})
            if 'content' in delta:
                print(delta['content'], end='', flush=True)
        
        # TGI 원본 형태 처리 (token 기반)
        elif 'token' in chunk:
            if not chunk.get('finished', False):
                print(chunk['token']['text'], end='', flush=True)
                
    except json.JSONDecodeError as e:
        # JSON 파싱 실패 시 건너뛰기 (또는 디버깅용 출력)
        # print(f"JSON 파싱 에러: {chunk_str[:100]}")  # 디버깅용
        continue
    except KeyError:
        # 예상하지 못한 키 구조는 건너뛰기
        continue
    except Exception as e:
        # 기타 예외는 건너뛰기
        # print(f"기타 에러: {e}")  # 디버깅용
        continue

print()  # 마지막에 줄바꿈

[05/25/25 08:47:42] INFO     Found credentials from IAM Role:                                   ]8;id=310336;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=306893;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

딥러닝은 인공지능의 한 분야로, 인공신경망을 사용하여 복잡한 데이터를 처리하고 학습하는 방법을 연구합니다. "딥"이라는 용어는 신경망의 레이어가 깊다는 의미로, 일반적인 신경망보다 더 많은 레이어를 가진 것을 의미합니다.

딥러닝의 핵심은 다음과 같습니다:

1. **신경망**: 인공신경망은 인공적으로 만들어진 신경망을 모델로 사용하여 데이터를 처리하고 학습합니다. 이는 인간의 뇌의 신경세포와 유사한 구조를 가지고 있습니다.

2. **학습**: 딥러닝 모델은 주어진 데이터에서 패턴을 찾아내고 이를 바탕으로 예측이나 분류를 수행합니다. 이 과정은 "학습"이라고 불리며, 모델이 정확도를 높이기 위해 데이터를 반복적으로 처리합니다.

3. **데이터**: 딥러닝은 대량의 데이터를 필요로 합니다. 모델이 정확한 예측을 할 수 있도록 하기 위해서는 다양한 데이터를 통해 모델이 다양한 상황을 이해할 수 있어야 합니다.

4. **적응성**: 딥러닝 모델은 새로운 데이터에 대해 적응할 수 있습니다. 이는 모델이 새로운 패턴을 학습하고 이를 바탕으로 예측을 개선할 수 있다는 것을 의미합니다.

딥러닝은 컴퓨터 비전, 자연어 처리, 음성 인식, 추천 시스템 등 다양한 분야에서 활용되고 있으며, 그 중요성은 점점 더 커지고 있습니다.


## 6. 리소스 정리

SageMaker HuggingFace Model 및 엔드포인트를 삭제 합니다.


In [ ]:
llm.delete_endpoint()

[05/25/25 08:47:57] INFO     Deleting endpoint configuration with name:                             ]8;id=813525;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=418973;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4995\4995]8;;\
                             sm-qwen-2-5-instruct-inf2-2025-05-25-08-32-51                                         

                    INFO     Deleting endpoint with name:                                           ]8;id=639900;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=598676;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4985\4985]8;;\
                             sm-qwen-2-5-instruct-inf2-2025-05-25-08-32-51                                         